<a href="https://colab.research.google.com/github/shabari12/AIML-Project-Series/blob/main/waterfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import median_absolute_error


In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.head()



,Index,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,0,<p>😅The pH value observed is: 0.2644088544860...,<br>😒With a hardness level of 2.6735383052025...,<br>😅The water sample exhibits a total solids ...,1.431972,1.303685,1.563001,0.697062,2.010126,<h1>😀Observing a turbidity value of 1.60887258...,1.252351
1,1,<p>😇The solution's pH level is 0.012034498894...,<h1>😀The water hardness is measured at 2.6303...,<br>😒The total solids content in the sample is...,0.911372,1.181754,1.741640,0.896108,1.807786,<br>😁The water sample exhibits a turbidity rea...,1.124565
2,2,<br>😀The pH reading measures 0.491167423175516...,<h2>😅With a hardness level of 1.2213782428674...,<h1>😍The water sample exhibits a total solids ...,1.506492,1.700871,2.322337,1.035440,1.548768,<p>😇Observing a turbidity value of 1.008907465...,0.525886
3,3,<h1>😅The water sample's pH level is 0.4865531...,<h2>😀With a hardness level of 1.2591335755542...,<h1>😅The total solids content in the sample is...,2.658852,3.035266,2.409255,1.185713,1.232375,<p>😇The water sample exhibits a turbidity read...,0.715288
4,4,<p>😍The observed pH value of 0.07548804288855...,<p>😁With a hardness level of 1.65681755735377...,<p>😍The total solids content in the sample is ...,2.169616,1.520531,1.990604,1.237182,1.589307,<h1>😒The water sample exhibits a turbidity rea...,0.890722


In [13]:
def clean_html_and_extract_values(cell):
    if isinstance(cell, str):
        no_html = re.sub(r'<[^>]+>', '', cell)
        numeric_values = re.findall(r'[-+]?\d*\.\d+|\d+', no_html)
        return " ".join(numeric_values) if numeric_values else no_html.strip()
    return cell

# Apply cleaning to both train and test data
train_data = train_data.applymap(clean_html_and_extract_values)
test_data = test_data.applymap(clean_html_and_extract_values)

<ipython-input-13-ebc8d224cdd5>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_data = train_data.applymap(clean_html_and_extract_values)
<ipython-input-13-ebc8d224cdd5>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data = test_data.applymap(clean_html_and_extract_values)


In [14]:
X = train_data.drop(columns=['Potability'])
y = train_data['Potability']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_val = X_val.apply(pd.to_numeric, errors='coerce')

In [16]:
X_train.fillna(X_train.mean(), inplace=True)
X_val.fillna(X_val.mean(), inplace=True)

In [17]:
print("Missing values after conversion:")
print(X_train.isnull().sum())


Missing values after conversion:
Index              0
ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
dtype: int64


In [18]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [28]:
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.preprocessing import MinMaxScaler


In [29]:
scaler_y = MinMaxScaler()

# Fit the scaler on the y_train values
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

# Build your model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model with the learning rate scheduler
model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32,
          validation_data=(X_val_scaled, y_val),
          callbacks=[LearningRateScheduler(lr_scheduler)])

# Make predictions
y_pred_scaled = model.predict(X_val_scaled)

# Inverse scale the predictions back to the original scale
y_pred_rescaled = scaler_y.inverse_transform(y_pred_scaled)

# Evaluate the model
r2 = r2_score(y_val, y_pred_rescaled)
medae = median_absolute_error(y_val, y_pred_rescaled)
print(f"R² Score: {r2}")
print(f"Median Absolute Error (MedAE): {medae}")

# Apply the custom score formula
score = max(0, 100 * (1 - medae))
print(f"Custom Score: {score}")

# Rescale the test data if needed
X_test = test_data.apply(pd.to_numeric, errors='coerce')
X_test.fillna(X_train.mean(), inplace=True)
X_test_scaled = scaler.transform(X_test)  # Make sure you scale the test data with the same scaler

# Make predictions on the test data
y_test_pred_scaled = model.predict(X_test_scaled)

# Inverse scale the test predictions
y_test_pred_rescaled = scaler_y.inverse_transform(y_test_pred_scaled)

# Create the final predictions DataFrame
predictions = pd.DataFrame(y_test_pred_rescaled, columns=['Potability'], index=test_data.index)
predictions['Index'] = test_data['Index']

# Print the results
print(predictions[['Index', 'Potability']].reset_index(drop=True).head())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.0370 - val_loss: 0.4963 - learning_rate: 0.0010
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0131 - val_loss: 0.4816 - learning_rate: 0.0010
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0117 - val_loss: 0.4668 - learning_rate: 0.0010
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0111 - val_loss: 0.4890 - learning_rate: 0.0010
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0106 - val_loss: 0.5023 - learning_rate: 0.0010
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0101 - val_loss: 0.4784 - learning_rate: 0.0010
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0100 - val_loss: 0.4918 - learning_rate: 0.0010
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0098 - val_loss: 0.5251 - learning_rate: 0.0010
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0097 - val_loss: 0.4745 - learning_rate: 0.0010


In [37]:
r2 = r2_score(y_val, y_pred_rescaled)
medae = median_absolute_error(y_val, y_pred_rescaled)
print(f"R² Score: {r2}")
score = max(0, 100 * (1 - medae))
print(f"Custom Score: {score}")

R² Score: 0.5553450392740139
Custom Score: 85.15198690836891


In [39]:
predictions = predictions[['Potability', 'Index']]
predictions.to_csv('predictionsfinal.csv', index=False)